Tugas Akhir

Nama : Muhamad Arif Adiputra

NIM : 13516114

(Penulisan catatan, kode, dan komentar program di bawah ini menggunakan bahasa Inggris karena kode dibuat dalam bahasa inggris)

----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 

Note: It will take normally around 2 minutes (at most 5 minutes) to load all necessary code. Please click run all button and wait while drink a cup of coffee (or anything)

#Load All Neccessary Code

Please give credits to Author if you are using all or part of all the code below

In [1]:
%%time
#I'm sorry I install it like this but I want it to run it directly from this notebook
!pip install pyarabic
!pip install numpy
!pip install pandas
!pip install gensim
!pip install nltk
!pip install regex
!pip install sklearn
!pip install matplotlib
!pip install seaborn
from IPython.display import clear_output
clear_output()

CPU times: user 162 ms, sys: 76.7 ms, total: 239 ms
Wall time: 11.4 s


In [2]:
%%time
from __future__ import print_function

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns # for statistical data visualization
%matplotlib inline

import nltk
import re

from sklearn.svm import SVC #Support Vector Machine
from sklearn.cluster import AffinityPropagation

# import metrics to compute accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import random

from IPython.display import clear_output
import time

import ipywidgets as widgets

import pyarabic.araby as araby
import pyarabic.number as number

from pyarabic.araby import strip_harakat
from pyarabic.araby import strip_tashkeel
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

CPU times: user 1.18 s, sys: 297 ms, total: 1.47 s
Wall time: 1.72 s


In [3]:
#@title Read Dataset
isRun = False #must always false, enable per code
isRunSVMExperiment = True
isChecking = True
isRunExperimentAfterStep6 = False

url_head = "https://raw.githubusercontent.com/futsuhotaru/qss/master/"
# Read dataset
df = pd.read_csv(url_head + "quran_simple_v2.csv")
df2 = pd.read_csv(url_head + "quran_root_words.csv")
df3 = pd.read_csv(url_head + "vector_list.csv")
df3_pred = pd.read_csv(url_head + "prediction_results/df3_pred.csv")

In [9]:
%%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

isAllListAlreadyCreated = False
if not(isAllListAlreadyCreated) :
    isUnique_root_listAlreadyCreated = False
    isWord_to_root_dictionaryAlreadyCreated = True
    isRoot_to_listwordsAlreadyCreated = False

isPrintWord_to_root_dictionaryInfo = False
isPrintDetailedInfo = False


### Word2Vec Build
### generate list of lists (bag of words)
# We are keeping the word as original as possible because we want to see the similar meaning and similar words to be having good score and 
# to be used with other dataset to compare the result and get the meaning and other things.
documents = []; count = 0
while(count<len(df)):
    documents.append(re.split('\s+', df.text[count]))
    count += 1
    
### Prepare model vocabulary. Variable Description for gensim.models.Word2Vec:
# size: size of vector; 
# window: how far to look before and after the word; 
# sg: fill 1 skip-gram, 0 for CBOW; 
# workers:number of worker threads to train model)
model = gensim.models.Word2Vec(size=500, window=100, min_count=2, sg=0, workers=10)
model.build_vocab(documents)  # prepare the model vocabulary

### Training
model.train(sentences=documents, total_examples=len(documents), epochs=model.iter)


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### CREATE unique_root_list
if(not(isUnique_root_listAlreadyCreated)):
    unique_root_list = []
    count = 0
    while(count<len(df3)):
        #for first case
        if(count==0):
            unique_root_list.append(df3.root[count])
        #other case
        elif not(df3.root[count] in unique_root_list):
            unique_root_list.append(df3.root[count])  
        count += 1
    isUnique_root_listAlreadyCreated = True

if((isUnique_root_listAlreadyCreated)):
    #this part is just for display
    print("length of unique_root_list: " + str(len(unique_root_list)))
    if (isPrintDetailedInfo):
        print("Sample data inside unique_root_list")
        index = 0; n_lines = len(unique_root_list); is_long = False
        if len(unique_root_list)>5:
            n_lines = 5
            is_long = True
        while(index<len(unique_root_list) and index<n_lines):
            print(str(index) + ": " + str(unique_root_list[index]))
            index += 1
        if is_long:print("...\n" + str(len(unique_root_list)-1) + ": " + str(unique_root_list[len(unique_root_list)-1]))

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### CREATE word_to_root_dictionary
#create list of list of unique arabic and root
if(not(isWord_to_root_dictionaryAlreadyCreated)):
    word_to_root_dictionary = []
    index = 0
    while(index<len(df3)):
        #for first case
        if(index==0):
            word_to_root_dictionary.append([df3.arabic[index], df3.root[index], df3.root_4[index], df3.root_3[index], df3.root_2[index], df3.root_1[index]])
        #other case
        else:
            isArabicAlreadyExist = False
            isRootAlreadyExist = False
            for wtr_list in word_to_root_dictionary:
                if df3.arabic[index] in wtr_list[0]:
                    isArabicAlreadyExist = True
                    break
                if df3.root[index] in wtr_list[1]:
                    isRootAlreadyExist = True
                    break

            if(not(isArabicAlreadyExist) or not(isRootAlreadyExist)):
                word_to_root_dictionary.append([df3.arabic[index], df3.root[index], df3.root_4[index], df3.root_3[index], df3.root_2[index], df3.root_1[index]]) 
        index += 1
    isWord_to_root_dictionaryAlreadyCreated = True

if((isPrintWord_to_root_dictionaryInfo)):
    #this part is just for display
    print("length of word_to_root_dictionary: " + str(len(word_to_root_dictionary)))
    if (isPrintDetailedInfo):
        print("Sample data inside word_to_root_dictionary")
        is_long = False
        index = 0; n_lines = len(word_to_root_dictionary)
        if len(word_to_root_dictionary)>5:
            n_lines = 5
            is_long = True
        while(index<len(word_to_root_dictionary) and index<n_lines):
            print(str(index) + ": " + str(word_to_root_dictionary[index]))
            index += 1
        if is_long: print("...\n" + str(len(word_to_root_dictionary)-1) + ": " + str(word_to_root_dictionary[len(word_to_root_dictionary)-1]))
        
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### CREATE root_to_listwords (words unique inside list)
#create list of list of arabic and root 
if(not(isRoot_to_listwordsAlreadyCreated)):
    root_to_listwords = []
    index = 0
    root_index = 0
    while(index<len(df3)):
        #for first case
        if(index==0):
            root_to_listwords.append([df3.root[index]])
            root_to_listwords[root_index].append([])
            root_to_listwords[root_index][1].append(df3.arabic[index])
        #other case
        else:
            isArabicAlreadyExist = False
            isRootAlreadyExist = False
            index_search_rootword = 0
            while index_search_rootword < len(root_to_listwords):
                if df3.root[index] in root_to_listwords[index_search_rootword][0]:
                    isRootAlreadyExist = True
                    if df3.arabic[index] in root_to_listwords[index_search_rootword][1]:
                        isArabicAlreadyExist = True
                    break
                index_search_rootword += 1
            if(not(isRootAlreadyExist) and not(isArabicAlreadyExist)):
                root_index += 1
                root_to_listwords.append([df3.root[index]])
                root_to_listwords[index_search_rootword].append([])
                root_to_listwords[index_search_rootword][1].append(df3.arabic[index])
            if(not(isArabicAlreadyExist) and isRootAlreadyExist):
                root_to_listwords[index_search_rootword][1].append(df3.arabic[index])

        index += 1
    isRoot_to_listwordsAlreadyCreated = True


if((isRoot_to_listwordsAlreadyCreated)):
    #this part is just for display
    print("length of root_to_listwords: " + str(len(root_to_listwords)))
    if (isPrintDetailedInfo):
        #root_to_listwords
        #check few first data
        print("Sample data inside root_to_listwords")
        is_long = False
        index = 0; n_lines = len(root_to_listwords)
        if len(root_to_listwords)>100:
            n_lines = 5
            is_long = True
        while(index<len(root_to_listwords) and index<n_lines):
            print(str(index) + ": " + str(root_to_listwords[index]))
            index += 1
        if is_long: print("...\n" + str(len(root_to_listwords)-1) + ": " + str(root_to_listwords[len(root_to_listwords)-1]))

    isAllListAlreadyCreated = True
    print("isAllListAlreadyCreated set to: " + str(isAllListAlreadyCreated))
    print()
    
isPrintDetailedInfo = False
isSVMListAlreadyCreated = False
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### CREATE root_to_listwords_svm_svm (words unique inside list)
#create list of list of arabic and root 
if(not(isSVMListAlreadyCreated)):
    root_to_listwords_svm = []
    index = 0
    root_index = 0
    while(index<len(df3_pred)):
        #for first case
        if(index==0):
            root_to_listwords_svm.append([df3_pred.y_svm_pred[index]])
            root_to_listwords_svm[root_index].append([])
            root_to_listwords_svm[root_index][1].append(df3_pred.arabic[index])
        #other case
        else:
            isArabicAlreadyExist = False
            isRootAlreadyExist = False
            index_search_rootword = 0
            while index_search_rootword < len(root_to_listwords_svm):
                if df3_pred.y_svm_pred[index] in root_to_listwords_svm[index_search_rootword][0]:
                    isRootAlreadyExist = True
                    if df3_pred.arabic[index] in root_to_listwords_svm[index_search_rootword][1]:
                        isArabicAlreadyExist = True
                    break
                index_search_rootword += 1
            if(not(isRootAlreadyExist) and not(isArabicAlreadyExist)):
                root_index += 1
                root_to_listwords_svm.append([df3_pred.y_svm_pred[index]])
                root_to_listwords_svm[index_search_rootword].append([])
                root_to_listwords_svm[index_search_rootword][1].append(df3_pred.arabic[index])
            if(not(isArabicAlreadyExist) and isRootAlreadyExist):
                root_to_listwords_svm[index_search_rootword][1].append(df3_pred.arabic[index])

        index += 1
    isSVMListAlreadyCreated = True

#this part is just for display
print("length of root_to_listwords_svm: " + str(len(root_to_listwords_svm)))
if (isPrintDetailedInfo):
    #root_to_listwords_svm
    #check few first data
    print("Sample data inside root_to_listwords_svm")
    is_long = False
    index = 0; n_lines = len(root_to_listwords_svm)
    if len(root_to_listwords_svm)>7:
        n_lines = 5
        is_long = True
    while(index<len(root_to_listwords_svm) and index<n_lines):
        print(str(index) + ": " + str(root_to_listwords_svm[index]))
        index += 1
    if is_long: print("...\n" + str(len(root_to_listwords_svm)-1) + ": " + str(root_to_listwords_svm[len(root_to_listwords_svm)-1]))

isSVMListAlreadyCreated = True
print("isSVMListAlreadyCreated set to: " + str(isSVMListAlreadyCreated))
print()

isPrintDetailedInfo = False
isAFPListAlreadyCreated = False

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### CREATE root_to_listwords_afp_svm (words unique inside list)
#create list of list of arabic and root 
if(not(isAFPListAlreadyCreated)):
    root_to_listwords_afp = []
    index = 0
    root_index = 0
    while(index<len(df3_pred)):
        #for first case
        if(index==0):
            root_to_listwords_afp.append([df3_pred.y_afp_pred[index]])
            root_to_listwords_afp[root_index].append([])
            root_to_listwords_afp[root_index][1].append(df3_pred.arabic[index])
        #other case
        else:
            isArabicAlreadyExist = False
            isRootAlreadyExist = False
            index_search_rootword = 0
            while index_search_rootword < len(root_to_listwords_afp):
                if df3_pred.y_afp_pred[index] in root_to_listwords_afp[index_search_rootword]:
                    isRootAlreadyExist = True
                    if df3_pred.arabic[index] in root_to_listwords_afp[index_search_rootword][1]:
                        isArabicAlreadyExist = True
                    break
                index_search_rootword += 1
            if(not(isRootAlreadyExist) and not(isArabicAlreadyExist)):
                root_index += 1
                root_to_listwords_afp.append([df3_pred.y_afp_pred[index]])
                root_to_listwords_afp[index_search_rootword].append([])
                root_to_listwords_afp[index_search_rootword][1].append(df3_pred.arabic[index])
            if(not(isArabicAlreadyExist) and isRootAlreadyExist):
                root_to_listwords_afp[index_search_rootword][1].append(df3_pred.arabic[index])

        index += 1
    isAFPListAlreadyCreated = True

#this part is just for display
print("length of root_to_listwords_afp: " + str(len(root_to_listwords_afp)))
if (isPrintDetailedInfo):
    #root_to_listwords_afp
    #check few first data
    print("Sample data inside root_to_listwords_afp")
    is_long = False
    index = 0; n_lines = len(root_to_listwords_afp)
    if len(root_to_listwords_afp)>7:
        n_lines = 5
        is_long = True
    while(index<len(root_to_listwords_afp) and index<n_lines):
        print(str(index) + ": " + str(root_to_listwords_afp[index]))
        index += 1
    if is_long: print("...\n" + str(len(root_to_listwords_afp)-1) + ": " + str(root_to_listwords_afp[len(root_to_listwords_afp)-1]))

isAFPListAlreadyCreated = True
print("isSVMListAlreadyCreated set to: " + str(isSVMListAlreadyCreated))
print()

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### All Function declaration
def Average(lst): 
    return sum(lst) / len(lst) 
# from https://www.tutorialspoint.com/python-program-to-sort-a-list-according-to-the-second-element-in-the-sublist
# Python program to sort the lists using the second element of sublist
# In place way to sort, use of third variable.
print("function now available: Average(lst) ")
def sortlist_ascending(A):
    l = len(A)
    for i in range(0, l):
        for j in range(0, l-i-1):
            if (A[j][1] > A[j + 1][1]):
                temporary = A[j]
                A[j]= A[j + 1]
                A[j + 1]= temporary
    return A
print("function now available: sortlist_ascending(A) ")
def sortlist_descending(A):
    l = len(A)
    for i in range(0, l):
        for j in range(0, l-i-1):
            if (A[j][1] < A[j + 1][1]):
                temporary = A[j]
                A[j]= A[j + 1]
                A[j + 1]= temporary
    return A
print("function now available: sortlist_descending(A) ")

# Driver Code
#A = [['AAA', 10], ['CCC', 15], ['RRRR', 2], ['XXXX', 150]]
#print(A)
#print(str(sortlist_ascending(A)))
#print(str(sortlist_descending(A)))
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Get index root position in root_to_listwords
def getRoot_index_reference_for_measurement(keyword):
    index_root = 0
    while index_root < len(root_to_listwords):
        if keyword in root_to_listwords[index_root][1]:
            break
        index_root += 1
    #print("root_index_reference_for_measurement: " + str(index_root))
    return(index_root)
print("function now available: getRoot_index_reference_for_measurement(keyword) ")

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Get index root position in root_to_listwords
def printRoot_index_reference_for_measurement(index_root):
    print("root_index_reference_for_measurement: " + str(index_root))
print("function now available: printRoot_index_reference_for_measurement(index_root) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Result from dictionary ()
def getResult_from_dictionary(keyword):
    result_from_dictionary = []
    result_dict_counter = 0

    index_root = 0
    while index_root < len(root_to_listwords):
        if keyword in root_to_listwords[index_root][1]:
            break
        index_root += 1

    for word in root_to_listwords[index_root][1]:
        result_dict_counter += 1
        cosine_similarity_value = model.wv.similarity(keyword, word)
        result_from_dictionary.append([word, cosine_similarity_value])

    sorted_rfd = sortlist_descending(result_from_dictionary)
    root_index_reference_for_measurement = index_root
    #print("result_from_dict: " + str(result_dict_counter))
    return(sorted_rfd)
print("function now available: getResult_from_dictionary(keyword) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Result from dictionary with topn ()
def getResult_from_dictionary_with_topn(keyword):
    result_from_dictionary_with_topn = model.wv.most_similar(positive=keyword, topn=10000)
    sorted_rfdt = result_from_dictionary_with_topn
    #print("result_from_dict_with_topn: " + str(len(sorted_rfdt)))
    return(sorted_rfdt)
print("function now available: getResult_from_dictionary_with_topn(keyword) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Result from svm ()
def getResult_from_svm(keyword):
    result_from_svm = []
    result_svm_counter = 0

    index_root = 0
    while index_root < len(root_to_listwords_svm):
        if keyword in root_to_listwords_svm[index_root][1]:
            break
        index_root += 1

    for word in root_to_listwords_svm[index_root][1]:
        result_svm_counter += 1
        cosine_similarity_value = model.wv.similarity(keyword, word)
        result_from_svm.append([word, cosine_similarity_value])

    sorted_rfs = sortlist_descending(result_from_svm)
    #print("result_from_svm: " + str(result_svm_counter))
    return(sorted_rfs)
print("function now available: getResult_from_svm(keyword) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Result from afp ()
def getResult_from_afp(keyword):
    result_from_afp = []
    result_afp_counter = 0

    index_root = 0
    while index_root < len(root_to_listwords_afp):
        if keyword in root_to_listwords_afp[index_root][1]:
            break
        index_root += 1

    for word in root_to_listwords_afp[index_root][1]:
        result_afp_counter += 1
        cosine_similarity_value = model.wv.similarity(keyword, word)
        result_from_afp.append([word, cosine_similarity_value])

    sorted_rfa = sortlist_descending(result_from_afp)

    #print("result_from_afp: " + str(result_afp_counter))
    return(sorted_rfa)
print("function now available: getResult_from_afp(keyword) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Evaluate Results
def evaluateAllResults(keyword, result_list): 
    # f1, p, r, tp, fp, fn
    all_evaluation_value = []
    #print("keyword: " + keyword)
    counter_result_list = 0
    for w2v_filter_result in result_list:

        tp = 0; fp = 0; fn = len(root_to_listwords[root_index_reference_for_measurement][1]) #tn not included because not relevant


        for tuple_item in w2v_filter_result:
            word = tuple_item[0]
            if word in root_to_listwords[root_index_reference_for_measurement][1] :
                tp += 1
                fn = fn - 1
            else :
                fp += 1

        p = tp /(tp+fp)
        r = tp /(tp+fn)
        f1 = tp/ (tp + 0.5*(fp+fn))
        
        all_evaluation_value.append([f1, p, r, tp, fp, fn])

        #print("F1:", str('{0:0.10f}'. format(f1)) + "; P:", str('{0:0.10f}'. format(p)) + "; R:", str('{0:0.10f}'. format(r))
        #      + " (using " + result_list_order[counter_result_list] + " with "
        #      + "tp:" + str(tp) + "; fp:" + str(fp) + "; fn:" + str(fn) + ")"
        #     )

        counter_result_list += 1
    
    return all_evaluation_value
print("function now available: evaluateAllResults(keyword, result_list) ")


def printEvaluateAllResults(all_evaluation_value):
    counter_result_list = 0
    for aev in all_evaluation_value:
        print("F1:", str('{0:0.10f}'. format(aev[0])) + "; P:", str('{0:0.10f}'. format(aev[1])) + "; R:", str('{0:0.10f}'. format(aev[2]))
              + " (using " + result_list_order[counter_result_list] + " with "
              + "tp:" + str(aev[3]) + "; fp:" + str(aev[4]) + "; fn:" + str(aev[5]) + ")"
             )
        counter_result_list += 1
print("function now available: printEvaluateAllResults(all_evaluation_value) ")

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### createResult_list
def createResult_list(keyword):
    sorted_rfd = getResult_from_dictionary(keyword)
    sorted_rfdt = getResult_from_dictionary_with_topn(keyword)
    sorted_rfs = getResult_from_svm(keyword)
    sorted_rfa = getResult_from_afp(keyword)
    result_list = [sorted_rfd, sorted_rfdt, sorted_rfs, sorted_rfa]
    return result_list
print("function now available: createResult_list(keyword) ")

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### printResult_list_information
def printResult_list_information(result_list):
    print("result_from_dict: " + str(len(result_list[0])))
    print("result_from_dict_with_topn: " + str(len(result_list[1])))
    print("result_from_svm: " + str(len(result_list[2])))
    print("result_from_afp: " + str(len(result_list[3])))
print("function now available: printResult_list_information(result_list) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### result_list_order
result_list_order = ["sorted_rfd", "sorted_rft", "sorted_rfs", "sorted_rfa"]
print("list now available: result_list_order ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### create_list_n_m_dim(n,m)
def create_list_n_m_dim(n,m):
    list_nm = []
    i=0; n=n; m=m
    while i < n:
        list_nm.append([])
        j = 0
        while j < m:
            list_nm[i].append([])
            j += 1
        i += 1
    return list_nm
print("function now available: create_list_n_m_dim(n,m) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### create_list_result_rearranged(list_ev_result_n_random_words)
def create_list_result_rearranged(list_ev_result_n_random_words):
    #rearrange list_ev_result_n_random_words so it can use average
    # algorithm, value, perwordvalue
    list_result_rearranged = create_list_n_m_dim(4,6)
    i_word = 0
    while i_word < len(list_ev_result_n_random_words):
        j_algorithm = 0
        while j_algorithm < len(list_ev_result_n_random_words[i_word]):
            k_value = 0
            while k_value < len(list_ev_result_n_random_words[i_word][j_algorithm]):
                list_result_rearranged[j_algorithm][k_value].append(list_ev_result_n_random_words[i_word][j_algorithm][k_value])
                k_value += 1
            j_algorithm += 1
        i_word += 1
    return list_result_rearranged
print("function now available: create_list_result_rearranged(list_ev_result_n_random_words) ")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### create_list_result_averaged_value(list_result_rearranged)
def create_list_result_averaged_value(list_result_rearranged):
    list_result_averaged_value = create_list_n_m_dim(4,6)
    i_algorithm = 0
    while i_algorithm < len(list_result_rearranged):
        j_value = 0
        while j_value < len (list_result_rearranged[i_algorithm]):
            #print(str(i_algorithm) + "; " + str(j_value))
            #print(str((list_result_rearranged[i_algorithm][j_value])))
            list_result_averaged_value[i_algorithm][j_value].append(Average(list_result_rearranged[i_algorithm][j_value]))
            j_value += 1
        i_algorithm += 1
    return list_result_averaged_value
print("function now available: create_list_result_averaged_value(list_result_rearranged) ")

def printEvaluateAllResultsFromList(list_result_averaged_value):
    counter_result_list = 0
    for lrav in list_result_averaged_value:
        print("F1:", str('{0:0.10f}'. format(lrav[0][0])) + "; P:", str('{0:0.10f}'. format(lrav[1][0])) + "; R:", str('{0:0.10f}'. format(lrav[2][0]))
              + " (using " + result_list_order[counter_result_list] + " with "
              + "tp:" + str(lrav[3][0]) + "; fp:" + str(lrav[4][0]) + "; fn:" + str(lrav[5][0]) + ")"
             )
        counter_result_list += 1
print("function now available: printEvaluateAllResults(all_evaluation_value) ")

keyword = ""
root_index_reference_for_measurement = 0

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Result from dictionary with topn ()
def getResult_from_dictionary_with_topn_value(keyword, topn_value):
    #print("getResult_from_dictionary_with_topn_value keyword: " + keyword)
    result_from_dictionary_with_topn = model.wv.most_similar(positive=keyword, topn=topn_value)
    sorted_rfdt = result_from_dictionary_with_topn
    #print("result_from_dict_with_topn: " + str(len(sorted_rfdt)))
    return(sorted_rfdt)
print("function now available: getResult_from_dictionary_with_topn(keyword) ")

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### Evaluate Results
def evaluateResult(keyword, result_list): 
    #print("evaluateResult keyword: " + keyword)
    # f1, p, r, tp, fp, fn
    all_evaluation_value = []
    #print("keyword: " + keyword)
    
    root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword)
    tp = 0; fp = 0; fn = len(root_to_listwords[root_index_reference_for_measurement][1]) #tn not included because not relevant
    for tuple_item in result_list:
        word = tuple_item[0]
        #print(tuple_item)
        if word in root_to_listwords[root_index_reference_for_measurement][1] :
            tp += 1
            fn = fn - 1
        else :
            fp += 1

    p = tp /(tp+fp)
    r = tp /(tp+fn)
    f1 = tp/ (tp + 0.5*(fp+fn))
    all_evaluation_value.append([f1, p, r, tp, fp, fn])
        
        
    return all_evaluation_value
print("function now available: evaluateResult(keyword, result_list) ")

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### printEvaluateResult
def printEvaluateResult(all_evaluation_value):
    #print("printEvaluateResult keyword: " + keyword)
    for aev in all_evaluation_value:
        print("F1:"+ str('{0:0.10f}'. format(aev[0])) + "; P:"+ str('{0:0.10f}'. format(aev[1])) + "; R:"+ str('{0:0.10f}'. format(aev[2]))
              + "; (with "
              + "tp:" + str(aev[3]) + "; fp:" + str(aev[4]) + "; fn:" + str(aev[5]) + ")"
             )
        
print("function now available: printEvaluateResult(all_evaluation_value) ")

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### printEvaluateResult
def strEvaluateResult(all_evaluation_value):
    #print("printEvaluateResult keyword: " + keyword)
    for aev in all_evaluation_value:
        result = ("F1:" + str('{0:0.10f}'. format(aev[0])) + "; P:" + str('{0:0.10f}'. format(aev[1])) + "; R:" + str('{0:0.10f}'. format(aev[2]))
                  + "; (with "
                  + "tp:" + str(aev[3]) + "; fp:" + str(aev[4]) + "; fn:" + str(aev[5]) + ")"
                 )
                 
    return result
    
print("function now available: strEvaluateResult(all_evaluation_value) ")

isKeywordRandom = False
if not(isKeywordRandom):
    keyword = "خَلَقَكُمْ"

if isKeywordRandom:
    keyword =  random.sample((random.sample(root_to_listwords, 1))[0][1], 1)[0]

print("keyword: " + keyword)
root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword)
#printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
result_list = createResult_list(keyword)

print()
#print("result:")
ev_result = evaluateAllResults(keyword, result_list)
#printEvaluateAllResults(ev_result)

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### isDisplaySearch_widget
isDisplaySearch_widget = widgets.ToggleButton(value=True,description='Show Word Result',disabled=False,button_style='')

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### isDisplayEvaluationMetrics_widget
isDisplayEvaluationMetrics_widget = widgets.ToggleButton(value=True,description='Show Evaluation Metrics',disabled=False,button_style='')

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### title_widget
title_widget = widgets.HTML(value="<h1> Semantic Search with Vector Space Model: Interactive GUI </h1>")


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### display_n_widget
display_n_widget = widgets.IntSlider(min=1, max=25, step=1, description='display_n:', value=10)
#display(display_n_widget)

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### algorithm_type_widget
algorithm_type_widget = widgets.ToggleButtons(
    options=['Reference', 'Word2Vector Only', 'SVM', 'Affinity Propagation'],
    description='Type:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Result Reference, Must have 100% value in all evaluation metrics', 'using word2vector only to get result', 'using SVM classification algorithm', 'using Affinity Propagation clustering algorithm'],
    #     icons=['check'] * 3
)

#algorithm_type_widget.disabled = True
#display(algorithm_type_widget)

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### keyword_widget
keyword_widget = widgets.Text(value='',placeholder='(insert word here)',description='String:',disabled=False)

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### search_btn_widget
search_btn_widget = widgets.Button(description='Search!')
#display(search_btn_widget)

def search_btn_widget_on_click(obj):
    global keyword
    global root_index_reference_for_measurement
    global result_list
    
    prev_btn_widget.disabled = False
    next_btn_widget.disabled = False
    reset_btn_widget.disabled = False
    random_words_widget.disabled = False
    clear_output(wait=True)
    display_all_widgets(title_widget, algorithm_type_widget, display_n_widget, keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget)
    if (isDisplaySearch_widget.value):
        keyword = keyword_widget.value
        #print(keyword)
        print("Showing Search Results:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)
        if algorithm_type_widget.value == 'Reference':
            printSearchResults(result_list, 0, start_point)
        elif algorithm_type_widget.value == 'Word2Vector Only':
            printSearchResults(result_list, 1, start_point)
        elif algorithm_type_widget.value == 'SVM':
            printSearchResults(result_list, 2, start_point)
        elif algorithm_type_widget.value == 'Affinity Propagation':
            printSearchResults(result_list, 3, start_point)
        else:
            print("error, no algorithm selected")
        
        print("----- ----- ----- ----- -----")
        print()
    
    if (isDisplayEvaluationMetrics_widget.value):
        keyword = keyword_widget.value
        #print(keyword)
        print("Showing Evaluation Metrics:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)

        print("result:")
        ev_result = evaluateAllResults(keyword, result_list)
        printEvaluateAllResults(ev_result)
        
        print("----- ----- ----- ----- -----")
        print()
        
    if (not(isDisplayEvaluationMetrics_widget.value) and not(isDisplaySearch_widget.value)):
        print("you didn't select anything to show!")
        
    
search_btn_widget.on_click(search_btn_widget_on_click)


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### next_btn_widget
next_btn_widget = widgets.Button(description='Next page', disabled=True)
#display(search_btn_widget)

def next_btn_widget_on_click(obj):
    global start_point
    start_point += display_n_widget.value
    
    clear_output(wait=True)
    display_all_widgets(title_widget, algorithm_type_widget, display_n_widget, keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget)
    if (isDisplaySearch_widget.value):
        keyword = keyword_widget.value
        print("Showing Search Results:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)
        if algorithm_type_widget.value == 'Reference':
            printSearchResults(result_list, 0, start_point)
        elif algorithm_type_widget.value == 'Word2Vector Only':
            printSearchResults(result_list, 1, start_point)
        elif algorithm_type_widget.value == 'SVM':
            printSearchResults(result_list, 2, start_point)
        elif algorithm_type_widget.value == 'Affinity Propagation':
            printSearchResults(result_list, 3, start_point)
        else:
            print("error, no algorithm selected")
        
        print("----- ----- ----- ----- -----")
        print()
    
    if (isDisplayEvaluationMetrics_widget.value):
        keyword = keyword_widget.value
        print("Showing Evaluation Metrics:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)

        print("result:")
        ev_result = evaluateAllResults(keyword, result_list)
        printEvaluateAllResults(ev_result)
        
        print("----- ----- ----- ----- -----")
        print()
        
    if (not(isDisplayEvaluationMetrics_widget.value) and not(isDisplaySearch_widget.value)):
        print("you didn't select anything to show!")
    
next_btn_widget.on_click(next_btn_widget_on_click)


##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### prev_btn_widget
prev_btn_widget = widgets.Button(description='Previous page', disabled=True)
#display(search_btn_widget)

def prev_btn_widget_on_click(obj):
    global start_point
    if (start_point - display_n_widget.value >= 0):
        start_point -= display_n_widget.value
    
    clear_output(wait=True)
    display_all_widgets(title_widget, algorithm_type_widget, display_n_widget, keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget)
    if (isDisplaySearch_widget.value):
        keyword = keyword_widget.value
        print("Showing Search Results:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)
        if algorithm_type_widget.value == 'Reference':
            printSearchResults(result_list, 0, start_point)
        elif algorithm_type_widget.value == 'Word2Vector Only':
            printSearchResults(result_list, 1, start_point)
        elif algorithm_type_widget.value == 'SVM':
            printSearchResults(result_list, 2, start_point)
        elif algorithm_type_widget.value == 'Affinity Propagation':
            printSearchResults(result_list, 3, start_point)
        else:
            print("error, no algorithm selected")
        
        print("----- ----- ----- ----- -----")
        print()
    
    if (isDisplayEvaluationMetrics_widget.value):
        keyword = keyword_widget.value
        print("Showing Evaluation Metrics:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)

        print("result:")
        ev_result = evaluateAllResults(keyword, result_list)
        printEvaluateAllResults(ev_result)
        
        print("----- ----- ----- ----- -----")
        print()
        
    if (not(isDisplayEvaluationMetrics_widget.value) and not(isDisplaySearch_widget.value)):
        print("you didn't select anything to show!")
    
prev_btn_widget.on_click(prev_btn_widget_on_click)

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### reset_btn_widget
reset_btn_widget = widgets.Button(description='Reset Page', disabled=True)
#display(search_btn_widget)

def reset_btn_widget_on_click(obj):
    global start_point
    start_point = 0
    
    clear_output(wait=True)
    display_all_widgets(title_widget, algorithm_type_widget, display_n_widget, keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget)
    if (isDisplaySearch_widget.value):
        keyword = keyword_widget.value
        print("Showing Search Results:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)
        if algorithm_type_widget.value == 'Reference':
            printSearchResults(result_list, 0, start_point)
        elif algorithm_type_widget.value == 'Word2Vector Only':
            printSearchResults(result_list, 1, start_point)
        elif algorithm_type_widget.value == 'SVM':
            printSearchResults(result_list, 2, start_point)
        elif algorithm_type_widget.value == 'Affinity Propagation':
            printSearchResults(result_list, 3, start_point)
        else:
            print("error, no algorithm selected")
        
        print("----- ----- ----- ----- -----")
        print()
    
    if (isDisplayEvaluationMetrics_widget.value):
        keyword = keyword_widget.value
        print("Showing Evaluation Metrics:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)

        print("result:")
        ev_result = evaluateAllResults(keyword, result_list)
        printEvaluateAllResults(ev_result)
        
        print("----- ----- ----- ----- -----")
        print()
        
    if (not(isDisplayEvaluationMetrics_widget.value) and not(isDisplaySearch_widget.value)):
        print("you didn't select anything to show!")
    
reset_btn_widget.on_click(reset_btn_widget_on_click)

##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### random_words_widget
random_words_widget = widgets.Button(description='Random Word', disabled=False)
#display(search_btn_widget)

def random_words_widget_on_click(obj):
    global keyword
    keyword_widget.value =  random.sample((random.sample(root_to_listwords, 1))[0][1], 1)[0]
    keyword = keyword_widget.value
    clear_output(wait=True)
    display_all_widgets(title_widget, algorithm_type_widget, display_n_widget, keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget)
    interact_manual(search_btn_widget_with_interact);

    
random_words_widget.on_click(random_words_widget_on_click)
        
        
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### printSearchResults
def printSearchResults(result_list, n, start_point):
    counter_result = 0
    #print(len(result_list[n]))
    for result in result_list[n]:
        printLinkString = "Links: https://quran.com/"
        if (counter_result>=start_point):
            i_location = 0; isFoundWithStep1 = False; isFoundWithStep2 = False; isFoundWithStep3 = False
            while i_location < len(df2):
                if result[0] == df2.arabic[i_location]:
                    isFoundWithStep1 = True
                    break 
                i_location += 1
            #print(str(i_location) + str(isFoundWithStep1))
            
            padString = ""
            n_stringlength = 15
            result_hatred = strip_tashkeel(strip_harakat(str([result[0]])))
            actualspace_result_length = len(result_hatred)
            i_space = 0
            while (i_space < n_stringlength - actualspace_result_length) :
                padString = padString + " "
                i_space += 1
            #print(str(actualspace_result_length) + ";" + str(i_space)+ ";" + str(n_stringlength))
            
            if isFoundWithStep1:
                stringStep = "with s1_search"
                printLinkString = printLinkString + str(df2.sura_num[i_location]) + "/" + str(df2.ayat_num[i_location]) + " or https://quran.kemenag.go.id/sura/" + str(df2.sura_num[i_location]) + "/" + str(df2.ayat_num[i_location])
                print( str("{:02d}".format(counter_result+1)) + ": " 
                      + str([result[0]]) + (padString)
                      + "( cos_value:" + str("{:.5f}".format(round(result[1], 5)))  + " root:" + str(df2.root[i_location])
                      + " meaning: '" + str(((df2.meaning[i_location]) + "'").ljust(20, ' ')) 
                      + " in sura_num:" + str(df2.sura_num[i_location]).ljust(3, ' ') + " and ayat_num:" + str(df2.ayat_num[i_location]) + " " + stringStep + " ) "
                      + "\n" + "                    " + printLinkString
                     )
            else :
                result_hat_stripped = strip_tashkeel(strip_harakat(result[0]))
                i_location = 0; 
                while i_location < len(df2):
                    df2_arabic_hat_stripped = strip_tashkeel(strip_harakat(df2.arabic[i_location]))
                    if result_hat_stripped == df2_arabic_hat_stripped:
                        isFoundWithStep2 = True
                        break  
                    i_location += 1
                    
                if isFoundWithStep2:
                    stringStep = "with s2_search"
                    printLinkString = printLinkString + str(df2.sura_num[i_location]) + "/" + str(df2.ayat_num[i_location]) + " or https://quran.kemenag.go.id/sura/" + str(df2.sura_num[i_location]) + "/" + str(df2.ayat_num[i_location])
                    print( str("{:02d}".format(counter_result+1)) + ": " 
                          + str([result[0]]) + (padString)
                          + "( cos_value:" + str("{:.5f}".format(round(result[1], 5)))  + " root:" + str(df2.root[i_location])
                          + " meaning: '" + str(((df2.meaning[i_location]) + "'").ljust(20, ' ')) 
                          + " in sura_num:" + str(df2.sura_num[i_location]).ljust(3, ' ') + " and ayat_num:" + str(df2.ayat_num[i_location]) + " " + stringStep + " ) "
                          + "\n" + "                    " + printLinkString
                         )
                
                else :
                    i_location = 0
                    while i_location < len(df):
                        if result[0] in df.text[i_location]:
                            isFoundWithStep3 = True
                            break
                        i_location += 1

                    if isFoundWithStep3:
                        stringStep = "with s3_search"
                        printLinkString = printLinkString + str(df2.sura_num[i_location]) + "/" + str(df2.ayat_num[i_location]) + " or https://quran.kemenag.go.id/sura/" + str(df2.sura_num[i_location]) + "/" + str(df2.ayat_num[i_location])
                        print( str("{:02d}".format(counter_result+1)) + ": " 
                              + str([result[0]]) + (padString)
                              + "( cos_value:" + str("{:.5f}".format(round(result[1], 5)))
                              + "' in sura_num:" + str(df.sura[i_location]) + " and ayat_num:" + str(df.aya[i_location]) + " " + stringStep + " ) "
                              + "\n" + "                    " + printLinkString
                             )
                                
                    else :
                        print( str("{:02d}".format(counter_result+1)) + ": "  
                              + str([result[0]]) 
                              + padString + "(location not found, fatal internal error)")

            
        if ((counter_result+1-start_point)==display_n_widget.value):
        #if counter_result == display_n_widget.value:
            break
        counter_result += 1
            
        
##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### ##### 
### display all widgets (with arrangements)
def display_all_widgets(title_widget, algorithm_type_widget, display_n_widget, keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget):
    
    row_title = widgets.HBox([title_widget]); display(row_title)
    row_1_widget_list = widgets.HBox([algorithm_type_widget, display_n_widget]); display(row_1_widget_list)
    row_2_widget_list = widgets.HBox([keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget]); display(row_2_widget_list)
    row_3_widget_list = widgets.HBox([search_btn_widget, random_words_widget, prev_btn_widget, reset_btn_widget, next_btn_widget]); display(row_3_widget_list)
    #row_4_widget_list = widgets.HBox(); display(row_4_widget_list)

clear_output(wait=True)

CPU times: user 27.2 s, sys: 1.15 s, total: 28.3 s
Wall time: 28.7 s


Run Main Window

In [10]:
#@title Main Window
start_point = 0
display_all_widgets(
    title_widget, 
    algorithm_type_widget, display_n_widget, 
    keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, 
    search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget
)

### here is sample test
#تُعْجِبْكَ arti general kagum
#قُلُوبَهُمْ arti general hati (kalbu)
#يَخْسِفَ arti menenggelamkan? (contoh kata sample 1)
#وَأَطِيعُونِ arti general patuh (di page 30 afp maknanya sama)
#مُصْبِحِينَ arti pagi
#سِمَانٍ
#تَنْزِيلًا
#وَلَدًا arti general laki laki
#رَجِيمٍ arti ditolak (sample with in give incorrect answer)
#رَجِيمٍ arti general dikutuk/ditolak (contoh hasil yang nilainya buruk di svm/afp)
#لِزَامًا hasil tidak ada arti (s3_search in ref, result 06 وَلِسُلَيْمَانَ)
#جُزْءًا portion/faction sama makna semantiknya namun akar katanya berbeda (01/05)
#غَوْرًا arti sunken (jatuh), contoh kata yang hasilnya berdekatan. Karena sampel kecil jadi hanya dapat ini

In [11]:
def search_btn_widget_with_interact():
    clear_output()
    global keyword
    global root_index_reference_for_measurement
    global result_list
    
    prev_btn_widget.disabled = False
    next_btn_widget.disabled = False
    reset_btn_widget.disabled = False
    random_words_widget.disabled = False
    clear_output(wait=True)
    #display_all_widgets(title_widget, algorithm_type_widget, display_n_widget, keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget)
    if (isDisplaySearch_widget.value):
        keyword = keyword_widget.value
        #print(keyword)
        print("Showing Search Results:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)
        if algorithm_type_widget.value == 'Reference':
            printSearchResults(result_list, 0, start_point)
        elif algorithm_type_widget.value == 'Word2Vector Only':
            printSearchResults(result_list, 1, start_point)
        elif algorithm_type_widget.value == 'SVM':
            printSearchResults(result_list, 2, start_point)
        elif algorithm_type_widget.value == 'Affinity Propagation':
            printSearchResults(result_list, 3, start_point)
        else:
            print("error, no algorithm selected")
        
        print("----- ----- ----- ----- -----")
        print()
    
    if (isDisplayEvaluationMetrics_widget.value):
        keyword = keyword_widget.value
        #print(keyword)
        print("Showing Evaluation Metrics:")
        root_index_reference_for_measurement = getRoot_index_reference_for_measurement(keyword); #printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        printRoot_index_reference_for_measurement(root_index_reference_for_measurement)
        result_list = createResult_list(keyword)

        print("result:")
        ev_result = evaluateAllResults(keyword, result_list)
        printEvaluateAllResults(ev_result)
        
        print("----- ----- ----- ----- -----")
        print()
        
    if (not(isDisplayEvaluationMetrics_widget.value) and not(isDisplaySearch_widget.value)):
        print("you didn't select anything to show!")

In [12]:
#@title Main Window
start_point = 0
display_all_widgets(
    title_widget, 
    algorithm_type_widget, display_n_widget, 
    keyword_widget, isDisplaySearch_widget, isDisplayEvaluationMetrics_widget, 
    search_btn_widget, prev_btn_widget, next_btn_widget, reset_btn_widget, random_words_widget
)
interact_manual(search_btn_widget_with_interact);


### here is sample test
#تُعْجِبْكَ arti general kagum
#قُلُوبَهُمْ arti general hati (kalbu)
#يَخْسِفَ arti menenggelamkan? (contoh kata sample 1)
#وَأَطِيعُونِ arti general patuh (di page 30 afp maknanya sama)
#مُصْبِحِينَ arti pagi
#سِمَانٍ
#تَنْزِيلًا
#وَلَدًا arti general laki laki
#رَجِيمٍ arti ditolak (sample with in give incorrect answer)
#رَجِيمٍ arti general dikutuk/ditolak (contoh hasil yang nilainya buruk di svm/afp)
#لِزَامًا hasil tidak ada arti (s3_search in ref, result 06 وَلِسُلَيْمَانَ)
#جُزْءًا portion/faction sama makna semantiknya namun akar katanya berbeda (01/05)
#غَوْرًا arti sunken (jatuh), contoh kata yang hasilnya berdekatan. Karena sampel kecil jadi hanya dapat ini

Showing Search Results:
01: ['تُعْلِنُونَ']     ( cos_value:1.00000 root:علن meaning: 'you reveal'          in sura_num:16  and ayat_num:19 with s1_search ) 
                    Links: https://quran.com/16/19 or https://quran.kemenag.go.id/sura/16/19
02: ['يُعْلِنُونَ']     ( cos_value:0.99189 root:علن meaning: 'they declare'        in sura_num:2   and ayat_num:77 with s1_search ) 
                    Links: https://quran.com/2/77 or https://quran.kemenag.go.id/sura/2/77
03: ['وَعَلَانِيَةً']    ( cos_value:0.99158 root:علن meaning: 'and openly'          in sura_num:2   and ayat_num:274 with s1_search ) 
                    Links: https://quran.com/2/274 or https://quran.kemenag.go.id/sura/2/274
----- ----- ----- ----- -----

Showing Evaluation Metrics:
result:
F1: 1.0000000000; P: 1.0000000000; R: 1.0000000000 (using sorted_rfd with tp:3; fp:0; fn:0)
F1: 0.0005971932; P: 0.0002987304; R: 0.6666666667 (using sorted_rft with tp:2; fp:6693; fn:1)
F1: 0.0594059406; P: 0.0306122449; R: 1.0